#1. Funciones auxiliares y lectura de datos


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

def leer_rango_excel_a_array(ruta_archivo, hoja_nombre=0, columnas_a_leer="", filas_a_leer=1):
    """
    Abre un archivo Excel y extrae un rango de celdas específico
    (D3 a BO42) y lo guarda en un array (lista de listas).
"""
    print(f"Intentando leer el archivo: {ruta_archivo}...")

    try:
        # 1. Filas:
        # Queremos empezar en la Fila 3.
        # Las filas en pandas se indexan desde 0.
        # Fila 1 = índice 0, Fila 2 = índice 1, Fila 3 = índice 2.
        # Así que saltamos las 2 primeras filas.
        filas_a_saltar = 3  # (Saltamos Fila 1 :texto Fila 2 : calibracion y Fila 3: checkeo salon vs dormitorio)


        #filas_a_leer = 40
        # Queremos leer desde la fila 3 hasta la 42 (inclusive).
        # Número de filas = 42 - 3 + 1 = 40 filas.

        # --- Lectura del archivo ---

        # Usamos 'header=None' porque el rango (D3) no es un encabezado,
        # es el inicio de los datos.
        df_rango = pd.read_excel(
            ruta_archivo,
            sheet_name= hoja_nombre, #Segunda hoja, Espectro Total
            header=None,             # Trata la primera fila leída (Fila 3) como datos
            skiprows=filas_a_saltar, # Salta las 2 primeras filas
            usecols=columnas_a_leer, # Lee solo el rango de columnas D-BO
            nrows=filas_a_leer       # Lee exactamente 40 filas
        )

        # Convertir el DataFrame de pandas a un array (lista de listas)
        # .values obtiene los datos como un array de NumPy
        # .tolist() lo convierte a una lista estándar de Python
        array_valores = df_rango.values.tolist()

        # Verificación
        if array_valores:
            num_filas = len(array_valores)
            num_cols = len(array_valores[0]) # Longitud de la primera fila
            print(f"Éxito: Se leyó un array de {num_filas} filas x {num_cols} columnas.")
        else:
            print("Advertencia: El rango D3:BO42 parece estar vacío.")

        return array_valores

    except FileNotFoundError:
        print(f"Error: No se pudo encontrar el archivo en la ruta: '{ruta_archivo}'.")
        return None
    except Exception as e:
        print(f"Ocurrió un error inesperado al leer el rango: {e}")
        return None

def separar_filas_en_matrices(datos_array):
    """
    Toma una lista de listas (matriz) y la separa en dos matrices:
    - Una con las filas de índice par (0, 2, 4...)
    - Otra con las filas de índice impar (1, 3, 5...)
    """

    # Listas para guardar las *filas* (que son listas)
    matriz_filas_impares_excel = []
    matriz_filas_pares_excel = []

    for i, fila in enumerate(datos_array):

        # i % 2 == 0 significa índice 0, 2, 4... (Filas Excel 3, 5, 7...)
        if i % 2 == 0:
            # --- CAMBIO CLAVE ---
            # Añade la 'fila' (que es una lista) como un nuevo elemento
            matriz_filas_impares_excel.append(fila)

        # else significa índice 1, 3, 5... (Filas Excel 4, 6, 8...)
        else:
            # --- CAMBIO CLAVE ---
            matriz_filas_pares_excel.append(fila)

    return matriz_filas_impares_excel, matriz_filas_pares_excel

def calcular_correccion_tonal(fase_espectro):
    # Aseguramos que sea un array de numpy
    arr = np.array(fase_espectro)
    
    # --- CÁLCULO DE DIFERENCIAS (Lt) ---
    centro = arr[:, 1:-1]
    izquierda = arr[:, :-2]
    derecha = arr[:, 2:]
    
    # Matriz (7, 28)
    Lt = centro - (izquierda + derecha) / 2

    # --- DIVISIÓN POR BANDAS ---
    # Lt_lo: (7, 9), Lt_mid: (7, 5), Lt_hi: (7, 14)
    Lt_lo, Lt_mid, Lt_hi = np.split(Lt, [9, 14], axis=1)

    # --- LÓGICA DE PENALIZACIÓN ---
    
    # 1. Calculamos el valor MÁXIMO de cada fila para cada banda
    max_lo  = Lt_lo.max(axis=1)
    max_mid = Lt_mid.max(axis=1)
    max_hi  = Lt_hi.max(axis=1)

    # 2. Condiciones globales por fase
    condicion_6 = (max_lo > 12) | (max_mid > 8) | (max_hi > 5)
    condicion_3 = (max_lo > 8)  | (max_mid > 5) | (max_hi > 3)

    # 3. Asignación de valores (Prioridad: 6 gana a 3, 3 gana a 0)
    resultado = np.where(condicion_6, 6, 
                np.where(condicion_3, 3, 0))

    T3 = np.r_[np.full(9, 8), np.full(5, 5), np.full(14, 3)]
    T6 = np.r_[np.full(9, 12), np.full(5, 8), np.full(14, 5)]

    # Matriz temporal donde marcamos con 3 o 6 quien supere el umbral
    P_map = (Lt > T3) * 3
    P_map[Lt > T6] = 6 

    # Buscamos el índice del máximo penalizador (+1 para ajustar a la matriz original)
    # Si la penalización es 0, devolvemos índice 0
    indices = np.where(resultado > 0, P_map.argmax(axis=1) + 1, 0)

    return resultado, indices

ABRIR ARCHIVOS, OBTENCIÓN DE LAeq SIN CORREGIR

In [2]:
#LO PRIMERO: ABRIR LOS ARCHIVOS

archivo_S1= "P3_G7_S1_Datos.xlsx"
archivo_S2= "P3_G7_S2_Datos.xlsx"

datos_sonometro_1 = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "U" , 24)
datos_sonometro_2 = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "U" , 48)

#SEPARAR POR PUNTOS A B Y C Y LUEGO RUIDOS DE FONDO

# Filtra los índices 3, 4, 5 de cada bloque de 6
datos_A = [x for i, x in enumerate(datos_sonometro_2) if i % 6 >= 3]

# Filtra los índices 0, 1, 2 de cada bloque de 6
datos_C = [x for i, x in enumerate(datos_sonometro_2) if i % 6 < 3]

ruido_A = np.array(datos_A[:3]) #QUITAR LAS TRES PRIMERAS POSICIONES QUE SON DE RUIDO DE FONDO
medidas_A = np.array(datos_A[3:])

ruido_B = np.array(datos_sonometro_1[:3]) #RUIDO EN B
medidas_B = np.array(datos_sonometro_1[3:]) #MEDIDAS EN B

ruido_C = np.array(datos_C[:3])
medidas_C = np.array(datos_C[3:])

#print(ruido_A, ruido_B, ruido_C) PARA COMPROBAR QUE FUNCIONE

#CORRECION RUIDO DE FONDO: PROMEDIADO DEL RUIDO DE FONDO ENERGETICO

ruido_A_promed = np.round(10 * np.log10((1/len(ruido_A)) * np.sum(10**(0.1*ruido_A))), 1)
ruido_B_promed = np.round(10 * np.log10((1/len(ruido_B)) * np.sum(10**(0.1*ruido_B))), 1)
ruido_C_promed = np.round(10 * np.log10((1/len(ruido_C)) * np.sum(10**(0.1*ruido_C))), 1)

#print(ruido_A_promed,ruido_B_promed, ruido_C_promed) #TUDO VA BEM

#print(len(medidas_A))
#AHORA VOY A SEPARAR LAS FASES

fases_A = np.split(medidas_A,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_B = np.split(medidas_B,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_C = np.split(medidas_C,7) #En este array están las medidas de cada fase, SIN PROMEDIAR


#print(fases_A[0]) Te devuelve los tres valores de la FASE 1 en la posición A

fases_A_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A
])

fases_B_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B
])

fases_C_promed = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C
])

#TODO: COMPROBAR EL CRITERIO EN CADA FASE DE QUE EL RUIDO DE FONDO SEA 3dB INFERIOR A LA MEDIDA DE LA FASE. SE REALIZA SOBRE EL PROMEDIADO DE LAS TRES MEDIDAS EN CADA PUNTO
#EJEMPLO : FASES_A_PROMED[0]-RUIDO_A_PROMED PARA COMPROBAR EN LA FASE 1
#A INVESTIGAR ANTES: ES POSIBLE QUE HAYA QUE HACER ANTES EL PROMEDIADO DE LOS PUNTOS ABC PARA COMPROBAR ESTO CONTRA EL RUIDO ABC? LA INTUICION ME DICE QUE NO, QUE ES PUNTO A PUNTO

#Comprobación de cada punto, cada fase si cumple el criterio de los 3dB

ruido_promed = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_promed)) + (10**(0.1*ruido_B_promed)) + (10**(0.1*ruido_C_promed)))), 1)
fases_promed = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed)) + (10**(0.1*fases_B_promed)) + (10**(0.1*fases_C_promed)))), 1)

diferencias = fases_promed - ruido_promed
print(diferencias)

#LAS FASES QUE NO CUMPLEN SON LAS 5, 6 Y 7. EN ELLAS NO SE PUEDE CORREGIR POR RUIDO DE FONDO Y LOS RESULTADOS SE DARÁN COMO COTAS

fases_1234 = fases_promed[:4]
fases_567 = fases_promed[4:7]

fases_1234_corregidas = np.round(10 * np.log10(((10**(0.1*fases_1234)) - (10**(0.1*ruido_promed)))), 1)

La_fases =  np.concatenate((fases_1234_corregidas , fases_567) , axis = 0)

print(fases_promed, La_fases)

Intentando leer el archivo: P3_G7_S1_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...
Éxito: Se leyó un array de 48 filas x 1 columnas.
[3.9 6.3 6.6 3.7 2.  0.5 0.9]
[43.3 45.7 46.  43.1 41.4 39.9 40.3] [41.  44.5 44.9 40.7 41.4 39.9 40.3]


CORRECION POR RUIDO BAJA FRECUENCIA

In [3]:
#Importamos la columna correspondiente a LCeq,T

datos_sonometro_1_LCeq = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "X" , 24)
datos_sonometro_2_LCeq = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "X" , 48)

datos_A_LCeq = [x for i, x in enumerate(datos_sonometro_2_LCeq) if i % 6 >= 3]


datos_C_LCeq = [x for i, x in enumerate(datos_sonometro_2_LCeq) if i % 6 < 3]

ruido_A_LCeq = np.array(datos_A_LCeq[:3])
medidas_A_LCeq = np.array(datos_A_LCeq[3:])

ruido_B_LCeq = np.array(datos_sonometro_1_LCeq[:3])
medidas_B_LCeq = np.array(datos_sonometro_1_LCeq[3:])

ruido_C_LCeq = np.array(datos_C_LCeq[:3])
medidas_C_LCeq = np.array(datos_C_LCeq[3:])


ruido_A_LCeq_promed = np.round(10 * np.log10((1/len(ruido_A_LCeq)) * np.sum(10**(0.1*ruido_A_LCeq))), 1)
ruido_B_LCeq_promed = np.round(10 * np.log10((1/len(ruido_B_LCeq)) * np.sum(10**(0.1*ruido_B_LCeq))), 1)
ruido_C_LCeq_promed = np.round(10 * np.log10((1/len(ruido_C_LCeq)) * np.sum(10**(0.1*ruido_C_LCeq))), 1)


fases_A_LCeq = np.split(medidas_A_LCeq,7)
fases_B_LCeq = np.split(medidas_B_LCeq,7)
fases_C_LCeq = np.split(medidas_C_LCeq,7)


fases_A_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A_LCeq
])

fases_B_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B_LCeq
])

fases_C_promed_LCeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C_LCeq
])

ruido_promed_LCeq = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_LCeq_promed)) + (10**(0.1*ruido_B_LCeq_promed)) + (10**(0.1*ruido_C_LCeq_promed)))), 1)
fases_promed_LCeq = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed_LCeq)) + (10**(0.1*fases_B_promed_LCeq)) + (10**(0.1*fases_C_promed_LCeq)))), 1)

print(ruido_promed_LCeq)
print(fases_promed_LCeq)

diferencias_LCeq = fases_promed_LCeq - ruido_promed_LCeq
print("Valores de LCeq - LAeq para cada fase de ruido: ",diferencias_LCeq) #NO CUMPLE NINGUNA, FULL COTAS

LCeq_fases = fases_promed_LCeq
print("Valores de LCeq con correccion de RF para cada fase de ruido: ",LCeq_fases)

Lf =  LCeq_fases - La_fases
print("Valores de Lf para cada fase de ruido: ",Lf)

Kf = [0,0,0,0,0,0,0] #TODOS LOS VALORES SUPERAN 15, POR TANTO TODAS LAS Kf SON 6 dB
print("Valores de Kf para cada fase de ruido: ",Kf)



Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 48 filas x 1 columnas.
60.1
[60.2 60.8 60.5 59.9 60.2 60.  59.8]
Valores de LCeq - LAeq para cada fase de ruido:  [ 0.1  0.7  0.4 -0.2  0.1 -0.1 -0.3]
Valores de LCeq con correccion de RF para cada fase de ruido:  [60.2 60.8 60.5 59.9 60.2 60.  59.8]
Valores de Lf para cada fase de ruido:  [19.2 16.3 15.6 19.2 18.8 20.1 19.5]
Valores de Kf para cada fase de ruido:  [0, 0, 0, 0, 0, 0, 0]


#CORRECCION POR IMPULSIVO

In [4]:
#Importamos la columna correspondiente a LAIeq,T

datos_sonometro_1_LAIeq = leer_rango_excel_a_array(archivo_S1, "Banda Ancha Total", "S" , 24)
datos_sonometro_2_LAIeq = leer_rango_excel_a_array(archivo_S2, "Banda Ancha Total", "S" , 48)



datos_A_LAIeq = [x for i, x in enumerate(datos_sonometro_2_LAIeq) if i % 6 >= 3]


datos_C_LAIeq = [x for i, x in enumerate(datos_sonometro_2_LAIeq) if i % 6 < 3]

ruido_A_LAIeq = np.array(datos_A_LAIeq[:3])
medidas_A_LAIeq = np.array(datos_A_LAIeq[3:])

ruido_B_LAIeq = np.array(datos_sonometro_1_LAIeq[:3])
medidas_B_LAIeq = np.array(datos_sonometro_1_LAIeq[3:])

ruido_C_LAIeq = np.array(datos_C_LAIeq[:3])
medidas_C_LAIeq = np.array(datos_C_LAIeq[3:])


ruido_A_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_A_LAIeq)) * np.sum(10**(0.1*ruido_A_LAIeq))), 1)
ruido_B_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_B_LAIeq)) * np.sum(10**(0.1*ruido_B_LAIeq))), 1)
ruido_C_LAIeq_promed = np.round(10 * np.log10((1/len(ruido_C_LAIeq)) * np.sum(10**(0.1*ruido_C_LAIeq))), 1)


fases_A_LAIeq = np.split(medidas_A_LAIeq,7)
fases_B_LAIeq = np.split(medidas_B_LAIeq,7)
fases_C_LAIeq = np.split(medidas_C_LAIeq,7)


fases_A_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_A_LAIeq
])

fases_B_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_B_LAIeq
])

fases_C_promed_LAIeq = np.array([
    np.round(10 * np.log10((1 / fase.size) * np.sum(10**(0.1 * fase))), 1)
    for fase in fases_C_LAIeq
])


ruido_promed_LAIeq = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_LAIeq_promed)) + (10**(0.1*ruido_B_LAIeq_promed)) + (10**(0.1*ruido_C_LAIeq_promed)))), 1)
fases_promed_LAIeq = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed_LAIeq)) + (10**(0.1*fases_B_promed_LAIeq)) + (10**(0.1*fases_C_promed_LAIeq)))), 1)

diferencias_LAIeq = fases_promed_LAIeq - ruido_promed_LAIeq

print("Valores de LAIeq - LAIeqRF para cada fase de ruido: ",diferencias_LAIeq) #CUMPLEN 1,2,3; NO CUMPLEN 4,5,6,7

fases_123 = fases_promed_LAIeq[:3]
fases_4567 = fases_promed_LAIeq[3:7]

fases_123_corregidas = np.round(10 * np.log10(((10**(0.1*fases_123)) - (10**(0.1*ruido_promed_LAIeq)))), 1)

LAIeq_fases =  np.concatenate((fases_123_corregidas , fases_4567) , axis = 0)
print("Valores de LAIeq con correccion de RF para cada fase de ruido: ",LAIeq_fases)

Li =  LAIeq_fases - La_fases
print("Valores de Li para cada fase de ruido: ",Li)

Ki = [0,3,3,0,0,0,0] #LOS QUE VALEN 6 ES POR LAS COTAS, EL RESTO ESTÁN TODOS POR DEBAJO DE 10, LUEGO SU Ki VALE 0
print("Valores de Ki para cada fase de ruido: ",Ki)

#TODO QUE ALGUIEN REVISE LO DE LAS COTAS, YO LE VEO SENTIDO PERO ECHADLE UN OJO POR CONTRASTAR

Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 1 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 48 filas x 1 columnas.
Valores de LAIeq - LAIeqRF para cada fase de ruido:  [ 5.2 13.3 13.4  2.9  0.9 -0.8 -0.2]
Valores de LAIeq con correccion de RF para cada fase de ruido:  [45.7 55.2 55.3 45.  43.  41.3 41.9]
Valores de Li para cada fase de ruido:  [ 4.7 10.7 10.4  4.3  1.6  1.4  1.6]
Valores de Ki para cada fase de ruido:  [0, 3, 3, 0, 0, 0, 0]


CORRECCION TONAL

In [5]:
Espectro_total_s1 = leer_rango_excel_a_array(archivo_S1, "Espectro Total", "EC:FF" , 24)
Espectro_total_s2 = leer_rango_excel_a_array(archivo_S2, "Espectro Total", "EC:FF" , 48)

datos_A_espectro = [x for i, x in enumerate(Espectro_total_s2) if i % 6 >= 3]
datos_C_espectro = [x for i, x in enumerate(Espectro_total_s2) if i % 6 < 3]

ruido_A_espectro = np.array(datos_A_espectro[:3])
medidas_A_espectro = np.array(datos_A_espectro[3:])

ruido_B_espectro = np.array(Espectro_total_s1[:3])
medidas_B_espectro = np.array(Espectro_total_s1[3:])

ruido_C_espectro = np.array(datos_C_espectro[:3])
medidas_C_espectro = np.array(datos_C_espectro[3:])

ruido_A_promed_espectro = np.round(10 * np.log10((1/len(ruido_A_espectro)) * np.sum(10**(0.1*ruido_A_espectro), axis = 0)), 1)
ruido_B_promed_espectro = np.round(10 * np.log10((1/len(ruido_B_espectro)) * np.sum(10**(0.1*ruido_B_espectro), axis = 0)), 1)
ruido_C_promed_espectro = np.round(10 * np.log10((1/len(ruido_C_espectro)) * np.sum(10**(0.1*ruido_C_espectro), axis = 0)), 1)

#print(ruido_A_promed,ruido_B_promed, ruido_C_promed) #TUDO VA BEM

#print(len(medidas_A))
#AHORA VOY A SEPARAR LAS FASES

fases_A_espectro = np.split(medidas_A_espectro,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_B_espectro = np.split(medidas_B_espectro,7) #En este array están las medidas de cada fase, SIN PROMEDIAR
fases_C_espectro = np.split(medidas_C_espectro,7) #En este array están las medidas de cada fase, SIN PROMEDIAR


#print(fases_A[0]) Te devuelve los tres valores de la FASE 1 en la posición A

fases_A_promed_espectro = np.array([
    np.round(10 * np.log10((1 / len(fase)) * np.sum(10**(0.1 * fase), axis = 0)), 1)
    for fase in fases_A_espectro
])

fases_B_promed_espectro = np.array([
    np.round(10 * np.log10((1 / len(fase)) * np.sum(10**(0.1 * fase), axis = 0)), 1)
    for fase in fases_B_espectro
])

fases_C_promed_espectro = np.array([
    np.round(10 * np.log10((1 / len(fase)) * np.sum(10**(0.1 * fase), axis = 0)), 1)
    for fase in fases_C_espectro
])

#TODO: COMPROBAR EL CRITERIO EN CADA FASE DE QUE EL RUIDO DE FONDO SEA 3dB INFERIOR A LA MEDIDA DE LA FASE. SE REALIZA SOBRE EL PROMEDIADO DE LAS TRES MEDIDAS EN CADA PUNTO
#EJEMPLO : FASES_A_PROMED[0]-RUIDO_A_PROMED PARA COMPROBAR EN LA FASE 1
#A INVESTIGAR ANTES: ES POSIBLE QUE HAYA QUE HACER ANTES EL PROMEDIADO DE LOS PUNTOS ABC PARA COMPROBAR ESTO CONTRA EL RUIDO ABC? LA INTUICION ME DICE QUE NO, QUE ES PUNTO A PUNTO

#Comprobación de cada punto, cada fase si cumple el criterio de los 3dB

ruido_promed_espectro = np.round(10 * np.log10((1/3) * ((10**(0.1*ruido_A_promed_espectro)) + (10**(0.1*ruido_B_promed_espectro)) + (10**(0.1*ruido_C_promed_espectro)))), 1)
fases_promed_espectro = np.round(10 * np.log10((1/3) * ((10**(0.1*fases_A_promed_espectro)) + (10**(0.1*fases_B_promed_espectro)) + (10**(0.1*fases_C_promed_espectro)))), 1)

diferencias_espectro = fases_promed_espectro - ruido_promed_espectro
print("La matriz de fases en espectro tiene la forma",{diferencias_espectro.shape})

num_filas, num_cols = fases_promed_espectro.shape
fases_corregidas_espectro = np.zeros((num_filas, num_cols))

num_filas, num_cols = fases_promed_espectro.shape
fases_corregidas_espectro = np.zeros((num_filas, num_cols))

for i in range(num_filas):
    # Fila actual de la fase (30 valores)
    fase_i = fases_promed_espectro[i]
    
    # CORRECCIÓN CLAVE AQUÍ: 
    # El ruido es el mismo vector para todas las fases. No usamos [i].
    ruido_i = ruido_promed_espectro 
    
    # Diferencia precalculada para esta fila
    diff_i = diferencias_espectro[i]
    
    # Copia base
    fila_corregida = fase_i.copy()
    
    # Índices a corregir en esta fila
    indices_a_corregir = np.where(diff_i > 3)[0]
    
    if len(indices_a_corregir) > 0:
        S = fase_i[indices_a_corregir]
        N = ruido_i[indices_a_corregir] # Ahora ruido_i es un vector, así que esto funciona
        
        # Fórmula
        potencia_lineal = (10**(0.1 * S)) - (10**(0.1 * N))
        potencia_lineal = np.maximum(potencia_lineal, 1e-10)
        resultado = np.round(10 * np.log10(potencia_lineal), 1)
        
        fila_corregida[indices_a_corregir] = resultado
    
    fases_corregidas_espectro[i, :] = fila_corregida

print("\nForma final corregida:", fases_corregidas_espectro.shape)
etiquetas_freq = np.array([
    "16", "20", "25", "31.5", "40", "50", "63", "80", "100", 
    "125", "160", "200", "250", "315", "400", "500", "630", "800", 
    "1k", "1.25k", "1.6k", "2k", "2.5k", "3.15k", "4k", "5k", "6.3k", 
    "8k", "10k", "12.5k"
])
#  REPORTE DE CUMPLIMIENTO (Bandas > 3dB)
print("\n--- Bandas VÁLIDAS por Fase ---")
for i in range(7):
    # Usamos máscara booleana para filtrar las etiquetas directamente
    validas = etiquetas_freq[diferencias_espectro[i] >= 3] 
    print(f"Fase {i+1} CUMPLE en: {list(validas)}")

Kt, posicion = calcular_correccion_tonal(fases_corregidas_espectro)

print(Kt)

posiciones_hz = etiquetas_freq[posicion]

print(posiciones_hz)


Intentando leer el archivo: P3_G7_S1_Datos.xlsx...
Éxito: Se leyó un array de 24 filas x 30 columnas.
Intentando leer el archivo: P3_G7_S2_Datos.xlsx...


/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/lib/python3/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Éxito: Se leyó un array de 48 filas x 30 columnas.
La matriz de fases en espectro tiene la forma {(7, 30)}

Forma final corregida: (7, 30)

--- Bandas VÁLIDAS por Fase ---
Fase 1 CUMPLE en: ['400', '500', '630', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k', '10k', '12.5k']
Fase 2 CUMPLE en: ['100', '250', '315', '400', '500', '630', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k', '10k', '12.5k']
Fase 3 CUMPLE en: ['200', '250', '315', '400', '500', '630', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k', '10k', '12.5k']
Fase 4 CUMPLE en: ['250', '315', '400', '500', '630', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k']
Fase 5 CUMPLE en: ['500', '800', '1k', '1.25k', '1.6k', '2k', '2.5k', '3.15k', '4k', '5k', '6.3k', '8k', '10k', '12.5k']
Fase 6 CUMPLE en: ['1.6k']
Fase 7 CUMPLE en: ['500']
[0 0 0 3 0 0 0]
['16' '16' '16' '1.6k' '16' '16' '16']


COMBINACIÓN DE TODO: HALLAMOS LK DE CADA FASE

In [6]:
print("Valores Laeq de cada fase" , La_fases)
print("Matriz de correcciones por ruido de baja frecuencia Kf:", Kf)
print("Matriz de correcciones por ruido impulsivo Ki:", Ki)
print("Matriz de correcciones por ruido tonal Kt:", Kt)


Lk_fases = La_fases + Kf + Ki + Kt #No tenemos que preocuparnos porque la suma de correciones sume más de 9 en ninguna fase

print("Matriz de LkeqTi en cada fase i:", Lk_fases)

print ("Las fases 5, 6 y 7 son cotas")

Valores Laeq de cada fase [41.  44.5 44.9 40.7 41.4 39.9 40.3]
Matriz de correcciones por ruido de baja frecuencia Kf: [0, 0, 0, 0, 0, 0, 0]
Matriz de correcciones por ruido impulsivo Ki: [0, 3, 3, 0, 0, 0, 0]
Matriz de correcciones por ruido tonal Kt: [0 0 0 3 0 0 0]
Matriz de LkeqTi en cada fase i: [41.  47.5 47.9 43.7 41.4 39.9 40.3]
Las fases 5, 6 y 7 son cotas


CALCULO DEL LK diurno, vespertino y nocturno

In [9]:
Lkeqd= 10*np.log10(1/12*((6*(10**(0.1*Lk_fases[0])))+(1*(10**(0.1*Lk_fases[1])))+(1*(10**(0.1*Lk_fases[2])))+(3*(10**(0.1*Lk_fases[3])))+(1*(10**(0.1*Lk_fases[4])))))

Lkeqe= 10*np.log10(1/4*((1*(10**(0.1*Lk_fases[2])))+(3*(10**(0.1*Lk_fases[3])))))

Lkeqn= 10*np.log10(1/8*((4.5*(10**(0.1*Lk_fases[0])))+(2.5*(10**(0.1*Lk_fases[5])))+(1*(10**(0.1*Lk_fases[6])))))

print("Valor Lkeq diurno:" , np.round(Lkeqd,1) ,"dBA")
print("Valor Lkeq vespertino:" , np.round(Lkeqe,1) ,"dBA")
print("Valor Lkeq nocturno:" , np.round(Lkeqn,1) ,"dBA")


Valor Lkeq diurno: 43.6 dBA
Valor Lkeq vespertino: 45.2 dBA
Valor Lkeq nocturno: 40.6 dBA
